In [1]:
import os
import torch
import bertviz
from bertviz import model_view, head_view
from bertviz.neuron_view import show
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys
sys.path.append('../') # Dirty hack
from train_transformers import get_base_transformer_lm, get_datasets_and_vocab
from vocabulary import CharVocabulary
from data_utils import build_dataset_addmult_mod10

from evaluate_transformers import package_data


In [3]:
MODEL = 'state_900000.pt'
MODEL_LOAD_PATH = os.path.join('/Users/kylemcgrath/projects/structural-grokking-330/sample_runs/', MODEL)

In [4]:
out_vocab = CharVocabulary(chars=set('0123456789'))
# datasets, in_vocab = get_datasets_and_vocab(, args.lm)

in_vocab = CharVocabulary(chars=set('0123456789+*()='))

vec_dim = 512
n_heads = 4
encoder_n_layers = 2

In [5]:
model, interface = get_base_transformer_lm(in_vocab, vec_dim, n_heads, encoder_n_layers, model_load_path=MODEL_LOAD_PATH)

INFO: Loading pretrained model from /Users/kylemcgrath/projects/structural-grokking-330/sample_runs/state_900000.pt


In [6]:
data_batch = ['(*(+(+41)(*40))(*7(*(+17)8)))', '(*(*(*(*4(+9(+73)))6)0)4)']

data_batch = ['(+(+4(*32))(*41))']

data_tensor = package_data(data_batch, in_vocab)
src = data_tensor['in'].T
src_len = data_tensor['in_len']

mask = model.generate_len_mask(src.shape[1], src_len)
src = model.pos_embed(model.input_embed(src), 0)
attn_matrices = model.trafo.get_attn_matrices(
    src, tgt=None, src_length_mask=mask
)
all_tokens = list(in_vocab.to_index.keys())


# Unsqueeze to have fake batch element

attn_matrices = tuple([i for i in attn_matrices])

print(attn_matrices[0].shape)


print(attn_matrices[0].shape)
print(data_batch)


torch.Size([1, 8, 17, 17])
torch.Size([1, 8, 17, 17])
['(+(+4(*32))(*41))']


/Users/kylemcgrath/projects/structural-grokking-330/notebooks/../collate.py:28: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = batch[0].storage()._new_shared(reduce(mul, s, 1))


In [7]:
model_view(attn_matrices, list(data_batch[0]))

<IPython.core.display.Javascript object>

In [8]:
head_view(attn_matrices, list(data_batch[0]))

<IPython.core.display.Javascript object>

In [11]:
def plot_attention_matrix(attention_matrix, tokens, head=None):
    """
    Plots the attention matrix for the model's heads.
    
    :param attention_matrix: A tensor of shape [num_heads, seq_len, seq_len]
                             representing the attention weights.
    :param tokens: A list of tokens corresponding to the sequence length.
    :param head: Optional; the specific head to visualize. If None, all heads are visualized.
    """
    num_heads = attention_matrix.shape[0]
    seq_len = attention_matrix.shape[1]

    if head is not None:
        heads_to_plot = [head]
    else:
        heads_to_plot = range(num_heads)

    fig, axes = plt.subplots(1, len(heads_to_plot), figsize=(15, 5))

    if len(heads_to_plot) == 1:
        axes = [axes]

    for i, ax in enumerate(axes):
        head_idx = heads_to_plot[i]
        ax.set_title(f'Head {head_idx}')

        # Convert the tensor to a numpy array after detaching it from the computation graph
        attn_data = attention_matrix[head_idx].detach().numpy()
        
        sns.heatmap(attn_data, ax=ax, cmap='viridis', xticklabels=tokens, yticklabels=tokens)
        ax.xaxis.set_tick_params(rotation=90)
        ax.yaxis.set_tick_params(rotation=0)

    plt.tight_layout()
    plt.show()

# Usage example
plot_attention_matrix(attn_matrices, list(data_batch)[0])

AttributeError: 'tuple' object has no attribute 'shape'

In [ ]:
plot_attention_matrix(attn_matrices, list)

In [ ]:
from transformers import AutoTokenizer, AutoModel, utils
from bertviz import model_view
utils.logging.set_verbosity_error()  # Suppress standard warnings

model_name = "microsoft/xtremedistil-l12-h384-uncased"  # Find popular HuggingFace models here: https://huggingface.co/models
input_text = "The cat sat on the mat"  
model = AutoModel.from_pretrained(model_name, output_attentions=True)  # Configure model to return attention values
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer.encode(input_text, return_tensors='pt')  # Tokenize input text
outputs = model(inputs)  # Run model
attention = outputs[-1]  # Retrieve attention from model outputs
tokens = tokenizer.convert_ids_to_tokens(inputs[0])  # Convert input ids to token strings
model_view(attention, tokens)  # Display model view

In [ ]:
attention[0].shape

In [ ]:
tokens